In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from datasets.dataset_dict import DatasetDict
from datasets import Dataset
import pandas as pd
%load_ext autoreload
%autoreload 2

data_path = "/data/desponds/data/Summarization/"

In [ ]:
from preprocessing import preprocessing_summarization
datasets, tokenized = preprocessing_summarization()

In [8]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'wb') as handle:
    pickle.dump(tokenized, handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'wb') as handle:
    pickle.dump(datasets, handle)

In [ ]:
for lang in ['fr','en'] :
    tokenized[lang] = tokenized[lang].map(lambda example: example, remove_columns=['summary', 'text', 'title'])

In [5]:
len(tokenized['fr']['train'][0]['input_ids'])

121

In [6]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'rb') as handle:
    tokenized = pickle.load(handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'rb') as handle:
    datasets = pickle.load(handle)

In [ ]:
from training import get_trainers_summarization
trainers = get_trainers_summarization(data_path, tokenized, langs = ['fr'])

In [ ]:
trainers['fr'].train()

In [ ]:
trainers['en'].train()

## Translation

In [1]:
import pickle
with open('/data/desponds/data/Summarization/tokenized.pickle', 'rb') as handle:
    tokenized = pickle.load(handle)
with open('/data/desponds/data/Summarization/datasets.pickle', 'rb') as handle:
    datasets = pickle.load(handle)

In [ ]:
from translation import translate_fr_en_summarization
translated_dataset = datasets['fr']['test'].select([5]).map(translate_fr_en_summarization, batched = True)
# with open('/data/desponds/data/Summarization/translated_dataset.pickle', 'wb') as handle:
#     pickle.dump(translated_dataset, handle)

## Inference

In [10]:
from transformers import pipeline

summarizer = pipeline("summarization", model="/data/desponds/data/Summarization/trainer_fr/checkpoint-38000")
summarizer(text)

[{'summary_text': '<extra_id_0>. Vous avez raison. »'}]